In [5]:
import cv2
from textwrap import wrap
import shutil
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow_addons as tfa
import os, warnings
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
import numpy as np
import random

print('TensorFlow Version ' + tf.__version__)

def seed_everything(seed = 0):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

seed_everything()

import warnings
warnings.filterwarnings("ignore")

TensorFlow Version 2.10.0


In [6]:
def data_augment(image):
    p_spatial = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    p_rotate = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    p_pixel_1 = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    p_pixel_3 = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    
    # Flips
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    
    if p_spatial > .75:
        image = tf.image.transpose(image)
        
    # Rotates
    if p_rotate > .75:
        image = tf.image.rot90(image, k = 3) # rotate 270º
    elif p_rotate > .5:
        image = tf.image.rot90(image, k = 2) # rotate 180º
    elif p_rotate > .25:
        image = tf.image.rot90(image, k = 1) # rotate 90º
        
    # Pixel-level transforms
    if p_pixel_1 >= .4:
        image = tf.image.random_saturation(image, lower = .7, upper = 1.3)
    if p_pixel_3 >= .4:
        image = tf.image.random_brightness(image, max_delta = .1)
        
    return image

In [7]:
image_size = 224
batch_size = 16
n_classes = 3
EPOCHS = 30

train_path = 'C:\\Users\\matus\\PycharmProjects\\VisionTransformer\\data\\Training Images'

classes = {0: "Dry",
           1: "Normal",
           2: "Wet"}
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255,
                                                          samplewise_center=True,
                                                          samplewise_std_normalization=True,
                                                          validation_split=0.2,
                                                          preprocessing_function=data_augment)
# set as training data

train_gen = datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=batch_size,
    seed=1,
    color_mode='rgb',
    shuffle=True,
    class_mode='categorical',
    subset='training')

# same directory as training data

valid_gen = datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=batch_size,
    seed=1,
    color_mode='rgb',
    shuffle=False,
    class_mode='categorical',
    subset='validation')

class_indices_mapping = train_gen.class_indice

Found 2707 images belonging to 3 classes.
Found 674 images belonging to 3 classes.


KeyboardInterrupt: 

In [8]:
x=np.concatenate([train_gen.next()[0] for i in range(train_gen.__len__())])
y=np.concatenate([train_gen.next()[1] for i in range(train_gen.__len__())])
print(x.shape)
print(y.shape)

(2707, 224, 224, 3)
(2707, 3)


In [17]:
X_train=x.reshape(2707,224*224*3)
X_train.shape

(2707, 150528)

In [18]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=2)
X_smote, y_smote = sm.fit_resample(X_train, y)

In [26]:
X_smote.shape
img = X_smote.reshape(7473,224,224,3)
img.shape

(7473, 224, 224, 3)

In [23]:
train_sep_dir = 'C:\\Users\\matus\\PycharmProjects\\VisionTransformer\\test_folder'

from keras.utils import array_to_img

pil_img = array_to_img(img[80] * 224)
pil_img.save('demo.jpg')


In [27]:
train_sep_dir = 'C:\\Users\\matus\\PycharmProjects\\VisionTransformer\\test_folder'

from keras.utils import array_to_img

save_dir = "test_smote_images"

# Create the directory if it doesn't exist
if not os.path.exists(train_sep_dir):
    os.mkdir(train_sep_dir)


#This function return label name 
def get_key(val):
    for key, value in class_indices_mapping.items():
        if val == value:
            return key
    return None  # Return None if the value is not found


# Loop through each image and label
for i in range(len(img)):
    label_index = np.argmax(y_smote[i])  # Get the index of the label with the highest value
    label = get_key(label_index)  # Get the label name from the index

    # Create a subdirectory for the class if it doesn't exist
    class_dir = os.path.join(save_dir, str(label))
    if not os.path.exists(class_dir):
        os.makedirs(class_dir)

    # Save the image with the corresponding label as the filename
    img_filename = f"smote_{i}.jpg"
    img_path = os.path.join(class_dir, img_filename)

    # Convert the image data back to PIL image and save
    pil_img = array_to_img(img[i] * 224)  # Scaling back to the original range
    pil_img.save(img_path)

    print(f"Image {i + 1}/{len(img)} saved as {img_path}")

Image 1/7473 saved as test_smote_images\Normal\smote_0.jpg
Image 2/7473 saved as test_smote_images\Normal\smote_1.jpg
Image 3/7473 saved as test_smote_images\Normal\smote_2.jpg
Image 4/7473 saved as test_smote_images\Normal\smote_3.jpg
Image 5/7473 saved as test_smote_images\Normal\smote_4.jpg
Image 6/7473 saved as test_smote_images\Normal\smote_5.jpg
Image 7/7473 saved as test_smote_images\Normal\smote_6.jpg
Image 8/7473 saved as test_smote_images\Normal\smote_7.jpg
Image 9/7473 saved as test_smote_images\Normal\smote_8.jpg
Image 10/7473 saved as test_smote_images\Normal\smote_9.jpg
Image 11/7473 saved as test_smote_images\Normal\smote_10.jpg
Image 12/7473 saved as test_smote_images\Dry\smote_11.jpg
Image 13/7473 saved as test_smote_images\Normal\smote_12.jpg
Image 14/7473 saved as test_smote_images\Normal\smote_13.jpg
Image 15/7473 saved as test_smote_images\Normal\smote_14.jpg
Image 16/7473 saved as test_smote_images\Normal\smote_15.jpg
Image 17/7473 saved as test_smote_images\Norma